In [2]:
from pyomo.environ import ConcreteModel, Objective, Expression, value, Var, Param, Constraint, ConstraintList, Set, assert_optimal_termination, \
check_optimal_termination, Block, log, log10, SolverFactory, units as pyunits
import pyomo.util.infeasible as infeas

from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, SolverFactory, TransformationFactory, units as pyunits, value


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

from IPython.display import clear_output

from sklearn.metrics import r2_score

import itertools

import plotly.express as px 
import ast
import scipy
import warnings
warnings.filterwarnings("ignore")

from multiprocessing import Process
from multiprocessing import Pool

import defs1

In [2]:
scenarios = ['ACCESS', 'NorESM', 'BCC-CS', 'CNRM-E', 'MPI-ES', 'MRI-ES']

In [14]:
for scenario_name in scenarios:
    print(scenario_name)
    hydro_df_final = pd.DataFrame()

    df_wbm = pd.read_csv(WBM RESULT OUTPUT FILE)

    df_adj = pd.read_csv("adj_df_final_combined_%s_nonlinear9.csv" % scenario_name)
    df_coeffs = pd.read_csv("final_combined_%s_nonlinear9.csv" % scenario_name)

    for eia in df_wbm.EIA_ID.unique():
        #print(eia)
        hydro_df = df_wbm[df_wbm.EIA_ID == eia]
        hydro_df['release_adjusted'] = hydro_df['Release (m3/s)'] * df_adj[df_adj.eia == eia].release_adj.max()
        hydro_df['storage_adjusted'] = hydro_df['Storage (m3)'] *  1e-6 * df_adj[df_adj.eia == eia].storage_adj.max()

        a = df_coeffs[df_coeffs.EIA_ID == eia].t1_a.max()
        b = df_coeffs[df_coeffs.EIA_ID == eia].t1_b.max()
        c = df_coeffs[df_coeffs.EIA_ID == eia].t1_c.max()
        d = df_coeffs[df_coeffs.EIA_ID == eia].t1_d.max()
        e = df_coeffs[df_coeffs.EIA_ID == eia].t1_e.max()
        f = df_coeffs[df_coeffs.EIA_ID == eia].t1_f.max()
        z = df_coeffs[df_coeffs.EIA_ID == eia].t1_z.max()

        hydro_df['generation'] = (a * hydro_df.release_adjusted +
                                  b * hydro_df.release_adjusted * hydro_df.storage_adjusted +
                                  c * hydro_df.storage_adjusted +
                                  e * hydro_df.release_adjusted ** d +
                                  z * hydro_df.storage_adjusted ** f)

        hydro_df_final = pd.concat([hydro_df_final,hydro_df], axis = 0, ignore_index=True)

    hydro_df_final.generation = np.where(hydro_df_final.CH_CW * 24 < hydro_df_final.generation, 
                                     hydro_df_final.CH_CW * 24, hydro_df_final.generation)
        
    hydro_df_final[['year', 'month', 'EIA_ID', 'CH_CW', 
                    'Release (m3/s)', 'Storage (m3)', 
                    'generation']].to_csv("hydropower_generation_%s.csv" % scenario_name)
    


ACCESS
NorESM
BCC-CS
CNRM-E
MPI-ES
MRI-ES
